# Marketing funnel analysis and optimization

### 1st part: Promotional spreadsheet

In [ ]:
import pulp as pl
import ipywidgets as widgets

planilha = pl.LpProblem('Custo_planilha', pl.LpMaximize)

# Variáveis 1a parte funil
total_mkt_plan = 3000 # input usuário
mensalidade_agencia = 4500  # input usuário
min_desp_var = 300 # input usuário
valor_planilha = 197    # input usuário
tx_plataforma = 0.1     # input usuário

despesa_plataforma = tx_plataforma * valor_planilha
pag_agencia =  mensalidade_agencia / 2  # mensalidade da agência diluida pelos membros do funil
inv_campanha = pl.LpVariable('inv_campanha', lowBound=0, cat='Continuous')  #d1
pg_agencia = pl.LpVariable('pg_agencia', lowBound=0, cat='Continuous')  #d2
desp_var = pl.LpVariable('desp_var', lowBound=0, cat='Continuous')  #d3
#taxa_plat = pl.LpVariable('taxa_plat', lowBound=0, cat='Continuous')  #d4

planilha += inv_campanha    # função objetivo (maximizar)

# Constraints
planilha += inv_campanha + pg_agencia + desp_var <= total_mkt_plan - despesa_plataforma, 'Budget_constraint'
planilha += desp_var >= min_desp_var, 'Desp_var_min'
planilha += pg_agencia >= pag_agencia, 'Pg_agencia_min'

planilha.solve()

if pl.LpStatus[planilha.status] == 'Optimal':
    for variable in planilha.variables():
        print("{} = {}".format(variable.name, variable.varValue))

# Potenciais compradores da planilha
cac_plan = 20    # custo de aquisição de clientes em reais (input usuário)
comp1 = inv_campanha.varValue / cac_plan
print('Número de potenciais compradores da planilha: ', str(comp1))

# Lucro na planilha
lucro_plan = comp1*(valor_planilha - despesa_plataforma) - total_mkt_plan
print('Lucro planilha: ', round(lucro_plan, 2))

desp_var = 300.0
inv_campanha = 430.3
pg_agencia = 2250.0
Número de potenciais compradores da planilha:  21.515
Lucro planilha:  814.61


### 2nd part: In-person event

In [ ]:
# Minimize total cost
evento = pl.LpProblem('Custo_evento', pl.LpMinimize)

d_min_local = 7000  # min de custos com local - input usuário
min_camp_evento = 1000  # min para campanha de eventos - input usuário

d_min_pub = 1000 # min de custos com publicidade - input usuário
d_max_pub = 2000 # max de custos com publicidade - input usuário

d_agencia = pag_agencia

d_min_inst = 1000  # min de custos com instrumentação - input usuário
d_max_inst = 3000  # min de custos com instrumentação - input usuário

d_min_buffet = 500     # despesa min com buffet - input usuário
d_max_buffet = 2000     # despesa max com buffet - input usuário

ingresso_evento = 700    # input usuário

conv1 = 0.15    # conversão planilha -> evento - input usuário (slider 0 a 100%)
cac_evento = 20     # input_usuário

d_total_evento_max = 18000   # max de custos com toda operação do evento - input usuário

inv_camp_evento = pl.LpVariable('inv_camp_evento', lowBound=min_camp_evento, cat='Continuous') # d5 -> alvo
d_buffet = pl.LpVariable('buffet', lowBound=d_min_buffet, upBound=d_max_buffet, cat='Continuous') # d2
d_pub_evento = pl.LpVariable('pub_evento', lowBound=d_min_pub, upBound=d_max_pub, cat='Continuous') # d3
d_inst = pl.LpVariable('instrumentacao', lowBound=d_min_inst, upBound=d_max_inst, cat='Continuous') # d4
d_local = pl.LpVariable('local', lowBound=d_min_local, cat='Continuous') # d1
d_total_evento = pl.LpVariable('total_desp', lowBound=0, upBound=d_total_evento_max, cat='Continuous')
#d_agencia = pl.LpVariable('agencia', lowBound=0, cat='Continuous') # d6

evento += d_total_evento   # objetivo

# Constraints
comp2 = conv1 * comp1
d_plataforma = tx_plataforma * ingresso_evento * comp2
evento += inv_camp_evento + d_buffet + d_pub_evento + d_inst + d_local <= d_total_evento - d_agencia - d_plataforma, 'Budget_constraint'

evento.solve()

print('Status da solução: ', pl.LpStatus[evento.status])

if pl.LpStatus[evento.status] == 'Optimal':
    for variable in evento.variables():
        print("{} = {}".format(variable.name, variable.varValue))

resultado_camp_evento = pl.value(inv_camp_evento) / cac_evento
compradores_evento = comp2 + resultado_camp_evento
receita_evento = compradores_evento*ingresso_evento

lucro_evento = receita_evento - pl.value(d_total_evento)

print('')
#print('Total de despesas lançamento: ', round(desp_lancamento, 2))
#print('Total despesas evento: ', round(total_desp_evento, 2))
print('Compradores evento: ', compradores_evento)
print('Compradores oriundos de campanha: ', resultado_camp_evento)
print('Compradores oriundos da planilha: ', comp2)
print("Total compras ingresso: ", round(receita_evento, 2))
print("Lucro evento: ", round(lucro_evento, 2))


Status da solução:  Optimal
buffet = 500.0
instrumentacao = 1000.0
inv_camp_evento = 1000.0
local = 7000.0
pub_evento = 1000.0
total_desp = 12975.907

Compradores evento:  53.22725
Compradores oriundos de campanha:  50.0
Compradores oriundos da planilha:  3.22725
Total compras ingresso:  37259.07
Lucro evento:  24283.17


In [ ]:
# Maximize ads investment -> maximize conversion
evento = pl.LpProblem('Custo_evento', pl.LpMaximize)

d_min_local = 7000  # min de custos com local - input usuário
min_camp_evento = 1000  # min para campanha de eventos - input usuário

d_min_pub = 1000 # min de custos com publicidade - input usuário
d_max_pub = 2000 # max de custos com publicidade - input usuário

d_agencia = pag_agencia

d_min_inst = 1000  # min de custos com instrumentação - input usuário
d_max_inst = 3000  # min de custos com instrumentação - input usuário

d_min_buffet = 500     # despesa min com buffet - input usuário
d_max_buffet = 2000     # despesa max com buffet - input usuário

ingresso_evento = 700    # input usuário

conv1 = 0.5    # conversão planilha -> evento - input usuário
cac_evento = 20     # input_usuário

d_total_evento_max = 18000   # max de custos com toda operação do evento - input usuário

inv_camp_evento = pl.LpVariable('inv_camp_evento', lowBound=min_camp_evento, cat='Continuous') # d5 -> alvo
d_buffet = pl.LpVariable('buffet', lowBound=d_min_buffet, upBound=d_max_buffet, cat='Continuous') # d2
d_pub_evento = pl.LpVariable('pub_evento', lowBound=d_min_pub, upBound=d_max_pub, cat='Continuous') # d3
d_inst = pl.LpVariable('instrumentacao', lowBound=d_min_inst, upBound=d_max_inst, cat='Continuous') # d4
d_local = pl.LpVariable('local', lowBound=d_min_local, cat='Continuous') # d1
d_total_evento = pl.LpVariable('total_desp', lowBound=0, upBound=d_total_evento_max, cat='Continuous')
#d_agencia = pl.LpVariable('agencia', lowBound=0, cat='Continuous') # d6

evento += inv_camp_evento   # objetivo

# Constraints
comp2 = conv1 * comp1
d_plataforma = tx_plataforma * ingresso_evento * comp2
evento += inv_camp_evento + d_buffet + d_pub_evento + d_inst + d_local <= d_total_evento - d_agencia - d_plataforma, 'Budget_constraint'

evento.solve()

print('Status da solução: ', pl.LpStatus[evento.status])

if pl.LpStatus[evento.status] == 'Optimal':
    for variable in evento.variables():
        print("{} = {}".format(variable.name, variable.varValue))

resultado_camp_evento = pl.value(inv_camp_evento) / cac_evento
compradores_evento = comp2 + resultado_camp_evento
receita_evento = compradores_evento*ingresso_evento

lucro_evento = receita_evento - pl.value(d_total_evento)

print('')
#print('Total de despesas lançamento: ', round(desp_lancamento, 2))
#print('Total despesas evento: ', round(total_desp_evento, 2))
print('Compradores evento: ', compradores_evento)
print('Compradores oriundos de campanha: ', resultado_camp_evento)
print('Compradores oriundos da planilha: ', comp2)
print("Total compras ingresso: ", round(receita_evento, 2))
print("Lucro evento: ", round(lucro_evento, 2))


Status da solução:  Optimal
buffet = 500.0
instrumentacao = 1000.0
inv_camp_evento = 5496.975
local = 7000.0
pub_evento = 1000.0
total_desp = 18000.0

Compradores evento:  285.60625
Compradores oriundos de campanha:  274.84875
Compradores oriundos da planilha:  10.7575
Total compras ingresso:  199924.38
Lucro evento:  181924.38


### 3rd part: Follow-up

In [ ]:
# Follow-up simulation

conv2 = 0.1     # conversão evento -> acompanhamento, input usuário (slider)
org = 10     # clientes do orgânico - input usuário
base_planejador = 800   # input usuário
d_comercial = 5000      # input usuário
n_planejadores = 2      # input usuário
valor_acompanhamento = 3000     # input usuário

n_clientes = conv2 * compradores_evento + org

receitas_acomp = n_clientes * valor_acompanhamento
desp_acomp = (d_comercial + n_clientes * base_planejador * n_planejadores)
lucro_acomp = receitas_acomp - desp_acomp
print("Lucro acompanhamento: ", lucro_acomp)

Lucro acompanhamento:  48984.875
